# MIB File Analysis for 4D STEM Data

This notebook analyzes MIB files to understand their structure, particularly for 4D STEM datasets.

## Overview
- MIB files contain header information followed by frame data
- Each frame has its own header + detector data
- For 4D STEM: frames correspond to scan positions, detector data is reciprocal space

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from pathlib import Path

# Import our MIB parsing functions
from src.mib_viewer.io.mib_loader import get_mib_properties, MibProperties

## Define Analysis Functions

In [ ]:
def analyze_mib_header(filepath):
    """Analyze MIB file header and extract key information"""
    print(f'Analyzing: {os.path.basename(filepath)}')
    print(f'File size: {os.path.getsize(filepath) / (1024**3):.2f} GB\n')
    
    # Read the main header (first 384 bytes)
    with open(filepath, 'rb') as f:
        header_bytes = f.read(384)
        f.seek(0, os.SEEK_END)
        filesize = f.tell()
    
    # Parse header as comma-separated values
    header_fields = header_bytes.decode().split(',')
    
    print('Header Fields (first 15):')
    for i, field in enumerate(header_fields[:15]):
        print(f'  [{i:2d}]: {field}')
    
    return header_fields, filesize

def analyze_frame_structure(header_fields, filesize):
    """Analyze frame structure and calculate dimensions"""
    # Parse header with existing function
    props = get_mib_properties(header_fields)
    
    print(f'\n=== Detector Properties ===')
    print(f'Detector size: {props.merlin_size[0]} x {props.merlin_size[1]} pixels')
    print(f'Pixel data type: {props.pixeltype}')
    print(f'Frame header size: {props.headsize} bytes')
    print(f'Dynamic range: {props.dyn_range}')
    print(f'Mode: {"Single" if props.single else "Quad"}')
    
    # Calculate frame structure
    detector_pixels = props.merlin_size[0] * props.merlin_size[1]
    pixel_size = props.pixeltype.itemsize
    data_size_per_frame = detector_pixels * pixel_size
    frame_size = props.headsize + data_size_per_frame
    
    print(f'\n=== Frame Structure ===')
    print(f'Pixels per frame: {detector_pixels:,}')
    print(f'Bytes per pixel: {pixel_size}')
    print(f'Data bytes per frame: {data_size_per_frame:,}')
    print(f'Total bytes per frame: {frame_size:,} (header + data)')
    
    # Calculate number of frames
    num_frames = filesize // frame_size
    
    print(f'\n=== Dataset Structure ===')
    print(f'Total frames: {num_frames:,}')
    
    return props, num_frames

def guess_scan_dimensions(num_frames):
    """Guess the most likely scan dimensions"""
    print(f'\n=== Scan Dimension Analysis ===')
    
    # Check for perfect square
    sqrt_frames = int(np.sqrt(num_frames))
    if sqrt_frames * sqrt_frames == num_frames:
        print(f'Perfect square: {sqrt_frames} x {sqrt_frames}')
        return (sqrt_frames, sqrt_frames)
    
    # Find all factor pairs
    factors = []
    for i in range(1, int(np.sqrt(num_frames)) + 1):
        if num_frames % i == 0:
            factors.append((i, num_frames // i))
    
    print('Possible rectangular scans:')
    for w, h in factors[-10:]:  # Show last 10 (largest factors)
        ratio = max(w, h) / min(w, h)
        print(f'  {w:3d} x {h:3d} (aspect ratio: {ratio:.2f})')
    
    # Find most square-like
    best_ratio = float('inf')
    best_dims = factors[-1]
    for w, h in factors:
        ratio = max(w, h) / min(w, h)
        if ratio < best_ratio:
            best_ratio = ratio
            best_dims = (w, h)
    
    print(f'\nMost square-like: {best_dims[0]} x {best_dims[1]} (ratio: {best_ratio:.2f})')
    return best_dims

## Analyze 4D STEM Files

In [ ]:
# Define file paths
data_dir = Path('/media/o2d/data/ORNL Dropbox/Ondrej Dyck/TEM data/2025/Andy 4D')
files = {
    'large': data_dir / '1_256x256_2msec_graphene.mib',
    'small': data_dir / '64x64 Test.mib'
}

# Check files exist
for name, path in files.items():
    if path.exists():
        print(f'{name}: {path.name} ✓')
    else:
        print(f'{name}: {path.name} ✗ (not found)')

### Large Dataset Analysis (256x256 scan)

In [ ]:
print('=' * 60)
print('LARGE DATASET ANALYSIS')
print('=' * 60)

if files['large'].exists():
    header_fields, filesize = analyze_mib_header(files['large'])
    props, num_frames = analyze_frame_structure(header_fields, filesize)
    scan_dims = guess_scan_dimensions(num_frames)
    
    # Calculate 4D array shape
    print(f'\n=== 4D Array Properties ===')
    shape_4d = (scan_dims[1], scan_dims[0], props.merlin_size[1], props.merlin_size[0])
    print(f'4D shape (sy, sx, qy, qx): {shape_4d}')
    
    # Memory requirements
    total_elements = np.prod(shape_4d)
    bytes_per_element = props.pixeltype.itemsize
    memory_gb = (total_elements * bytes_per_element) / (1024**3)
    print(f'Total data points: {total_elements:,}')
    print(f'Memory requirement: {memory_gb:.2f} GB')
else:
    print('Large file not found!')

### Small Dataset Analysis (64x64 scan)

In [ ]:
print('\n' + '=' * 60)
print('SMALL DATASET ANALYSIS')
print('=' * 60)

if files['small'].exists():
    header_fields, filesize = analyze_mib_header(files['small'])
    props, num_frames = analyze_frame_structure(header_fields, filesize)
    scan_dims = guess_scan_dimensions(num_frames)
    
    # Calculate 4D array shape
    print(f'\n=== 4D Array Properties ===')
    shape_4d = (scan_dims[1], scan_dims[0], props.merlin_size[1], props.merlin_size[0])
    print(f'4D shape (sy, sx, qy, qx): {shape_4d}')
    
    # Memory requirements
    total_elements = np.prod(shape_4d)
    bytes_per_element = props.pixeltype.itemsize
    memory_gb = (total_elements * bytes_per_element) / (1024**3)
    print(f'Total data points: {total_elements:,}')
    print(f'Memory requirement: {memory_gb:.2f} GB')
else:
    print('Small file not found!')

## Load and Visualize Sample Data

Let's load a small portion of the data to understand its structure.

In [ ]:
def load_sample_frames(filepath, num_sample_frames=4):
    """Load a few sample frames to understand data structure"""
    print(f'Loading {num_sample_frames} sample frames from {os.path.basename(filepath)}')
    
    # Get file properties
    with open(filepath, 'rb') as f:
        header_fields = f.read(384).decode().split(',')
    
    props = get_mib_properties(header_fields)
    
    # Create memory-mapped array for just the sample frames
    merlin_frame_dtype = np.dtype([
        ('header', np.bytes_, props.headsize),
        ('data', props.pixeltype, props.merlin_size)
    ])
    
    # Load sample frames
    sample_data = np.memmap(
        filepath,
        dtype=merlin_frame_dtype,
        mode='r',
        shape=(num_sample_frames,)
    )
    
    return sample_data, props

# Load sample from small dataset (more manageable)
if files['small'].exists():
    sample_data, props = load_sample_frames(files['small'], 4)
    
    print(f'Sample data shape: {sample_data.shape}')
    print(f'Frame data shape: {sample_data["data"][0].shape}')
    print(f'Data type: {sample_data["data"].dtype}')
    
    # Show statistics for first frame
    first_frame = sample_data['data'][0]
    print(f'\nFirst frame statistics:')
    print(f'  Min: {first_frame.min()}')
    print(f'  Max: {first_frame.max()}')
    print(f'  Mean: {first_frame.mean():.2f}')
    print(f'  Std: {first_frame.std():.2f}')

In [ ]:
# Visualize the sample frames
if 'sample_data' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    axes = axes.ravel()
    
    for i in range(4):
        frame = sample_data['data'][i]
        im = axes[i].imshow(frame, cmap='viridis')
        axes[i].set_title(f'Frame {i} (Diffraction Pattern)')
        axes[i].axis('off')
        plt.colorbar(im, ax=axes[i])
    
    plt.tight_layout()
    plt.show()
    
    # Show sum of all sample frames (should show average diffraction pattern)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Sum
    sum_pattern = np.sum(sample_data['data'], axis=0)
    im1 = ax1.imshow(sum_pattern, cmap='viridis')
    ax1.set_title('Sum of Sample Frames')
    plt.colorbar(im1, ax=ax1)
    
    # Log scale
    im2 = ax2.imshow(np.log1p(sum_pattern), cmap='viridis')
    ax2.set_title('Sum of Sample Frames (Log Scale)')
    plt.colorbar(im2, ax=ax2)
    
    plt.tight_layout()
    plt.show()

## Performance Considerations

Based on the analysis above, let's calculate the performance requirements and chunking strategies.

In [ ]:
def analyze_performance_requirements(shape_4d, dtype):
    """Analyze memory and performance requirements for 4D dataset"""
    sy, sx, qy, qx = shape_4d
    bytes_per_element = dtype.itemsize
    
    print(f'4D Dataset: {sy} × {sx} × {qy} × {qx}')
    print(f'Data type: {dtype} ({bytes_per_element} bytes/pixel)')
    
    # Total memory
    total_elements = sy * sx * qy * qx
    total_gb = (total_elements * bytes_per_element) / (1024**3)
    print(f'\nTotal dataset: {total_gb:.2f} GB')
    
    # Slice sizes
    real_space_slice = qy * qx * bytes_per_element  # Single diffraction pattern
    recip_space_slice = sy * sx * bytes_per_element  # Single k-space pixel
    
    print(f'\nSlice sizes:')
    print(f'  Single diffraction pattern: {real_space_slice / 1024:.1f} KB')
    print(f'  Single reciprocal space slice: {recip_space_slice / 1024:.1f} KB')
    
    # Chunking recommendations
    target_chunk_mb = 10  # Target 10MB chunks
    elements_per_chunk = (target_chunk_mb * 1024 * 1024) // bytes_per_element
    
    # Real space chunking (good for virtual imaging)
    frames_per_chunk = elements_per_chunk // (qy * qx)
    chunk_sy = min(sy, int(np.sqrt(frames_per_chunk)))
    chunk_sx = min(sx, frames_per_chunk // chunk_sy)
    
    print(f'\nRecommended chunking for HDF5:')
    print(f'  Target chunk size: {target_chunk_mb} MB')
    print(f'  Real-space chunks: ({chunk_sy}, {chunk_sx}, {qy}, {qx})')
    
    # Memory requirements for different operations
    print(f'\nMemory for common operations:')
    print(f'  Load 1 diffraction pattern: {real_space_slice / 1024:.1f} KB')
    print(f'  Load 1 real-space line: {sx * qy * qx * bytes_per_element / (1024**2):.1f} MB')
    print(f'  Load 1 recip-space line: {sy * sx * bytes_per_element / 1024:.1f} KB')
    print(f'  Virtual dark field image: {sy * sx * bytes_per_element / 1024:.1f} KB')

# Analyze both datasets if available
if 'props' in locals():
    print('SMALL DATASET PERFORMANCE:')
    analyze_performance_requirements((64, 64, props.merlin_size[1], props.merlin_size[0]), props.pixeltype)
    
    print('\n' + '='*50)
    print('LARGE DATASET PERFORMANCE:')
    analyze_performance_requirements((256, 256, props.merlin_size[1], props.merlin_size[0]), props.pixeltype)

## Summary and Next Steps

Based on this analysis, we can now plan the 4D viewer implementation:

1. **Data Structure**: 4D arrays with shape (scan_y, scan_x, detector_y, detector_x)
2. **Memory Management**: Use HDF5 with chunking for datasets > 4GB
3. **Visualization**: Multiple linked 2D views with real-time slicing
4. **Performance**: PyQtGraph for real-time updates, downsampling during interaction